<a href="https://colab.research.google.com/github/kake01/Dialogue/blob/Preprocessing/%E5%90%8D%E5%8F%A4%E5%B1%8B%E5%A4%A7%E5%AD%A6%E3%81%AE%E5%89%8D%E5%87%A6%E7%90%86_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#名古屋大コーパスの前処理,ベクトル化

## 環境構築

In [0]:
# pip install janome

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
import numpy as np
import zipfile
import pickle
# from janome.tokenizer import Tokenizer

## データセットの解凍と前処理と保存

###データの解凍

In [0]:
with zipfile.ZipFile('data.zip') as existing_zip:
  existing_zip.extractall('data')

###データの前処理

In [0]:
text = ""
# s = "私は柴犬【しばいぬ】とシャム猫【しゃむねこ】が大好きです。"
# s = re.sub("【[^】]+】", "", s)  # 【と】の間に】以外の文字が複数ある箇所を、空の文字列に置き換える
for data_num in range(129):
  # ファイルデータを1つずつ読み込む
  with open("data/nucc/data_"+str(data_num+1)+".txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    texts = f.readline()
    while texts:
      # 1行ずつ読み込む
      texts = re.sub("《[^》]+》", "", texts)  # ルビの削除
      texts = re.sub("［[^］]+］", "", texts)  # 読みの注意の削除
      texts = re.sub("〔[^〕]+〕", "", texts)  # 読みの注意の削除
      texts = re.sub("＜[^＞]+＞", "", texts)  # ＜＞の削除
      texts = re.sub("（[^）]+）", "", texts)  # ())の削除
      texts = re.sub("＠[^\n]+\n", "", texts)  # @から始まる文を削除
      # texts = re.sub("[ 　\n「」『』（）｜※＊…]", "", texts)  # 全角半角スペース、改行、その他記号の削除
      texts = re.sub("[ 　「」『』（）｜※＊…]", "", texts)  # 全角半角スペース、改行、その他記号の削除

      if '：' in texts:
        # 文頭に名前が書いてある場合のみ名前の削除を行う
         texts = re.sub("^[^：]+：", "", texts, flags=re.MULTILINE)   #^で行頭を探せる
      
      text += texts
      texts = f.readline()
f.close()

###データの保存

In [0]:
# コーパスとして保存する
with open("corpus.txt", mode="w", encoding="utf-8") as f:
    f.write(text)

##コーパスのサイズ確認

In [0]:
with open('corpus.txt') as f:
  texts = f.read()
print("文字数:", len(texts))
# print(texts[:200])

In [0]:
# ファイルの読み込み
with open("corpus.txt", mode="r", encoding="utf-8") as f:
  text = f.read()
seperator = "。"  # 。をセパレータに指定
sentence_list = text.split(seperator)  # セパレーターを使って文章をリストに分割する
sentence_list.pop() # 最後の要素は空の文字列になるので、削除
sentence_list = [x+seperator for x in sentence_list]  # 文章の最後に。を追加


t = Tokenizer()

# words = []
# for sentence in sentence_list:
#     words.append(t.tokenize(sentence, wakati=True))   # 文章ごとに単語に分割し、リストに格納
    
# with open('words.pickle', mode='wb') as f:  # pickleに保存
#     pickle.dump(words, f)

##単語のベクトル化(word2vec)

###単語のベクトル化を行う

In [0]:
from gensim.models import word2vec

# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 0)

In [0]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

###類似語のテストを行う

In [0]:
print(model.wv.most_similar("怖い"))  # 最も似ている単語

###モデル,行列の保存を行う

In [0]:
# # モデルを保存
model.save("model.model")
# モデルを読み込む
# model = word2vec.Word2Vec.load("model.model")

with open('model.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(model.wv.vectors, f)

##辞書の作成

In [0]:
chars = []

for char in text:  # コーパスに使われている文字を追加
    if char not in chars:
        chars += char
        
chars += "\t\n"  # タブと改行を追加
chars_list = sorted(list(chars))  # 文字列をリストに変換してソートする
# print(chars_list)


with open("temp.pickle", mode="wb") as f:  # pickleで保存
    pickle.dump(chars_list, f)

# インデックスと文字で辞書を作成
char_indices = {}  # 文字がキーでインデックスが値
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}  # インデックスがキーで文字が値
for i, char in enumerate(chars_list):
    indices_char[i] = char

print(indices_char)

##encoder,decoderへの入力,decoderの正解を作成
TODO:word2vec使って行う

In [0]:
# max_sentence_length = 128  # 文章の最大長さ。これより長い文章はカットされる。
# sentence_list = [sentence for sentence in sentence_list if len(sentence) <= max_sentence_length]  # 長すぎる文章のカット

# n_char = len(chars_list)  # 文字の種類の数
# n_sample = len(sentence_list) - 1  # サンプル数

# x_sentences = []  # 入力の文章
# t_sentences = []  # 正解の文章
# for i in range(n_sample):
#     x_sentences.append(sentence_list[i])
#     t_sentences.append("\t" + sentence_list[i+1] + "\n")  # 正解は先頭にタブ、末尾に改行を加える
# max_length_x = max_sentence_length  # 入力文章の最大長さ
# max_length_t = max_sentence_length + 2  # 正解文章の最大長さ

# x_encoder = np.zeros((n_sample, max_length_x, n_char), dtype=np.bool)  # encoderへの入力
# x_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool)  # decoderへの入力
# t_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool)  # decoderの正解

# for i in range(n_sample):
#     x_sentence = x_sentences[i]
#     t_sentence = t_sentences[i]
#     for j, char in enumerate(x_sentence):
#         x_encoder[i, j, char_indices[char]] = 1  # encoderへの入力をone-hot表現で表す
#     for j, char in enumerate(t_sentence):
#         x_decoder[i, j, char_indices[char]] = 1  # decoderへの入力をone-hot表現で表す
#         if j > 0:  # 正解は入力より1つ前の時刻のものにする
#             t_decoder[i, j-1, char_indices[char]] = 1
            
# print(x_encoder.shape)